<a href="https://colab.research.google.com/github/HonahX/iceberg-summit-workshop/blob/colab_dev/Iceberg_getting_started_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iceberg Workshop: Getting Started

## How to run this workshop

The workshop is consisted of several code cells that are designed to be executed from top to bottom.

For example, this is the a code cell contains code to print "Hello Iceberg Summit"


In [1]:
print("Hello Iceberg Summit")

Hello Iceberg Summit


To execute a cell, click it and press Shift + Enter. The output will be displayed below the cell.

To execute a cell, click it and press Shift + Enter. The output will be displayed below the cell.

# Iceberg Metadata Structure

![My Image](https://github.com/HonahX/iceberg-summit-workshop/blob/main/notebooks/imgs/iceberg-metadata.png?raw=true)

# Setup

## Install Dependencies

In [ ]:
%pip install pyiceberg[pyarrow,pandas,sql-sqlite]==0.9.0

## Create utils to print directory

In [3]:
import os

def print_directory(root_path, indent=''):
    try:
        entries = sorted(os.listdir(root_path))
    except FileNotFoundError:
        print(f"{indent}[Error] Path not found: {root_path}")
        return
    except PermissionError:
        print(f"{indent}[Error] Permission denied: {root_path}")
        return

    for i, entry in enumerate(entries):
        path = os.path.join(root_path, entry)
        is_last = (i == len(entries) - 1)
        branch = '└── ' if is_last else '├── '
        print(f"{indent}{branch}{entry}")
        if os.path.isdir(path):
            new_indent = indent + ('    ' if is_last else '│   ')
            print_directory(path, new_indent)

## Download Example Data

In [4]:
import os
data_dir = "/data"
os.makedirs(data_dir, exist_ok=True)

!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet -O /data/yellow_tripdata_2024-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet -O /data/yellow_tripdata_2024-02.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet -O /data/yellow_tripdata_2024-03.parquet

--2025-04-06 16:10:04--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.160.201.126, 18.160.201.5, 18.160.201.131, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.160.201.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49961641 (48M) [binary/octet-stream]
Saving to: ‘/data/yellow_tripdata_2024-01.parquet’

/data/yellow_tripda 100%[===================>]  47.65M   144MB/s    in 0.3s    

2025-04-06 16:10:04 (144 MB/s) - ‘/data/yellow_tripdata_2024-01.parquet’ saved [49961641/49961641]

--2025-04-06 16:10:04--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.160.201.126, 18.160.201.5, 18.160.201.131, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.160.201.126|:443... connec

## Setup Catalog

In [ ]:
from pyiceberg.catalog import load_catalog


warehouse = "/warehouse"

sqlite_uri = f"sqlite:////{warehouse}/sql-catalog.db"
catalog = load_catalog("in-memory", warehouse=warehouse, **{
    "uri": "sqlite:///:memory:"
})

catalog.create_namespace_if_not_exists("demo_ns")

# Cleanup To Ensure Re-runnable

In [ ]:
try:
    # In case the table already exists
    catalog.drop_table("demo_ns.nyc_taxis")
except:
    pass

## Example Data: NYC Taxi Dataset

In this workshop, we will use New York City Taxi & Limousine Commission's Trip Record Data, which can be downloaded from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
import pyarrow.parquet as pq

taxis_data_1 = pq.read_table('/data/yellow_tripdata_2024-01.parquet')
taxis_data_2 = pq.read_table('/data/yellow_tripdata_2024-02.parquet')
taxis_data_3 = pq.read_table('/data/yellow_tripdata_2024-03.parquet')
dataset_schema = taxis_data_1.schema
dataset_schema

## Create an Iceberg table

First, we'll create an iceberg table using the dataset's schema.

In [ ]:
TABLE_NAME = "demo_ns.nyc_taxis"

In [ ]:
nyc_taxis_tbl = catalog.create_table(TABLE_NAME, schema=dataset_schema)
nyc_taxis_tbl

## What happens behind table creation?

A metadata file has been created and registered as the latest metadata of table `demo_ns.nyc_taxis`. Let's view the table's location.

In [ ]:
print_directory(nyc_taxis_tbl.location())

# Add data to the table

It will create a new snapshot on the table

In [ ]:
nyc_taxis_tbl.append(taxis_data_1)
nyc_taxis_tbl

## Read the table

We can see example data has been added to the table

In [ ]:
nyc_taxis_tbl.scan(limit=10).to_pandas()

## What happens when adding data?

The data has been written into a parquet file and a new snapshot has been created.

Let's check the table location again:

In [ ]:
print_directory(nyc_taxis_tbl.location())

In the `metadata`, we can see some new files are generated:


*   new metadata file: `00001-<uuid>-.metadata.json`
*   manifest file: `<uuid>-m0.avro`
*   manifest list file: `snap-<snapshot-id>-0-<uuid>.avro`

In the `data`, we can see a new parquet file that contains the inerted data



*   `00000-0-<uuid>.parquet`



# Table Evolution: Make table partitioned

The table we just created is unpartitioned. In this example, we want to take a further step to partition the table. We will partition the table by the `day` value of`tpep_pickup_datatime` column.

In [ ]:
from pyiceberg.transforms import DayTransform

with nyc_taxis_tbl.update_spec() as update_spec:
    update_spec.add_field("tpep_pickup_datetime", DayTransform())

nyc_taxis_tbl

## Insert new data

The newly inserted data will be partitioned by the `day` value of `tpep_pickup_datetime` column

In [ ]:
nyc_taxis_tbl.append(taxis_data_2)

In [ ]:
nyc_taxis_tbl.scan(limit=3).to_pandas()

In [ ]:
nyc_taxis_tbl.scan().to_pandas().size

# Partitioned Data

If we go to the `data` folder of the table, we can see the newly inserted data partitioned by date.

In [ ]:
print_directory(os.path.join(nyc_taxis_tbl.location(), "data"))

## Table Evolution: Change to partition by month for future data insertion

I changed my mind and now I want to partition the table by the "month" of `tpep_pickup_datetime` for any furture data insertion. No worries—we can easily achieve it!

Iceberg allows you to update the partitioning strategy without recreating the table or re-writing any data.

In [ ]:
from pyiceberg.transforms import MonthTransform

with nyc_taxis_tbl.update_spec() as update_spec:
    update_spec.remove_field("tpep_pickup_datetime_day")
    update_spec.add_field("tpep_pickup_datetime", MonthTransform())

nyc_taxis_tbl

Now let's append some new data to the table

In [ ]:
nyc_taxis_tbl.append(taxis_data_3)

If we go to the the `data` folder of table `nyc_taxis` again, we will find the new data is partitioned by the month value. (You can find folders of new partitions at the bottom)

```
├── tpep_pickup_datetime_month=2002-12
│   └── 00000-2-<uuid>.parquet
├── tpep_pickup_datetime_month=2024-02
│   └── 00000-1-<uuid>.parquet
├── tpep_pickup_datetime_month=2024-03
│   └── 00000-0-<uuid>.parquet
└── tpep_pickup_datetime_month=2024-04
    └── 00000-3-<uuid>.parquet
```

In [ ]:
print_directory(os.path.join(nyc_taxis_tbl.location(), "data"))

# Table Evolution: Change Table Schema
Iceberg supports schema evolution without rewriting any data. For example, we can rename `VendorId` to `ID`.


In [ ]:
# Before rename
nyc_taxis_tbl.scan(limit=3).to_pandas()

In [ ]:
with nyc_taxis_tbl.update_schema() as update:
    update.rename_column("VendorID", "ID")

In [ ]:
# After rename
nyc_taxis_tbl.scan(limit=3).to_pandas()

# Metadata Table

We can get more details of an iceberg by looking at its metadata tables.

## Partitions
For example, to learn about existing partitions in the table, we can query the `partitions` metadata table

In [ ]:
nyc_taxis_tbl.inspect.partitions().to_pandas()

## Files

If we want to see all the data files in the table, we can query the `files` metadata table

In [ ]:
nyc_taxis_tbl.inspect.files().to_pandas()

## Snapshots

If we want to look at snapshots of the table, we can query the `snapshots` metadata table.

Every time when a data change operation happens, Iceberg will form a new snapshot. In this example, we did 3 append and therefore we will have 3 snapshots

In [ ]:
nyc_taxis_tbl.inspect.snapshots().to_pandas()

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
  .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
  .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.8.1,org.apache.iceberg:iceberg-aws-bundle:1.8.1")
  .config("spark.sql.catalog.demo.type", "jdbc")
  .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
  .config("spark.sql.catalog.demo.uri", "jdbc:sqlite:///:memory:")
  .config("spark.sql.catalog.demo.warehouse", warehouse)
).getOrCreate()

In [ ]:
spark.sql("SELECT ID, tpep_pickup_datetime, fare_amount FROM demo.demo_ns.nyc_taxis LIMIT 5").show()